# Deep Recurrent Q-Network for VizDoom


In [1]:
import numpy as np
import random
import tensorflow as tf
import matplotlib.pyplot as plt
import scipy.misc
import os
import csv
import itertools
import tensorflow.contrib.slim as slim
%matplotlib inline

from helper2 import *

### Load the game environment

In [2]:
from vizdoom import *
a_size = 3 # Agent can move Left, Right, or Fire
image_size = 84

use_other_buffers = False #Wheter we use the depth buffer and label buffer instead of the screen buffer
use_RGB = False #Whether we use GRB or black and white
if use_other_buffers == True:
    image_chls = 2
else:
    if use_RGB == True:
        image_chls = 3
    else:
        image_chls = 1

#The Below code is related to setting up the Doom environment
game = DoomGame()
game.set_doom_scenario_path("defend_the_center.wad")  #This corresponds to the simple task we will pose our agent
game.load_config("defend_the_center.cfg")
game.set_doom_map("map01")
game.set_screen_resolution(ScreenResolution.RES_160X120)

if use_RGB == True:
    game.set_screen_format(ScreenFormat.RGB8)
else:
    game.set_screen_format(ScreenFormat.GRAY8)
    
game.set_render_hud(False)
game.set_render_crosshair(False)
game.set_render_weapon(True)
game.set_render_decals(False)
game.set_render_particles(False)

#Enable other usefull buffers for test purpose
game.set_depth_buffer_enabled(True)
game.set_automap_buffer_enabled(True)
game.set_labels_buffer_enabled(True)

game.add_available_button(Button.TURN_LEFT)
game.add_available_button(Button.TURN_RIGHT)
game.add_available_button(Button.ATTACK)
actions_list = np.identity(a_size,dtype=bool).tolist()
print(actions_list)

game.add_available_game_variable(GameVariable.AMMO2)
game.add_available_game_variable(GameVariable.POSITION_X)
game.add_available_game_variable(GameVariable.POSITION_Y)
game.set_episode_timeout(300)
game.set_episode_start_time(0)
game.set_window_visible(False)
game.set_sound_enabled(False)
#game.set_living_reward(-1)
game.set_mode(Mode.PLAYER)
game.init()

#End Doom set-up

env = game

[[True, False, False], [False, True, False], [False, False, True]]


### Implementing the network itself

In [3]:
class Qnetwork():
    def __init__(self,h_size,rnn_cell,myScope):
        #The network recieves a frame from the game, flattened into an array.
        #It then resizes it and processes it through four convolutional layers.
        
            
        self.scalarInput =  tf.placeholder(shape=[None,image_size * image_size * image_chls],dtype=tf.float32)
        self.imageIn = tf.reshape(self.scalarInput,shape=[-1,image_size,image_size,image_chls])
        self.conv1 = slim.convolution2d( \
            inputs=self.imageIn,num_outputs=32,\
            kernel_size=[8,8],stride=[4,4],padding='VALID', \
            biases_initializer=None,scope=myScope+'_conv1')
        self.conv2 = slim.convolution2d( \
            inputs=self.conv1,num_outputs=64,\
            kernel_size=[4,4],stride=[2,2],padding='VALID', \
            biases_initializer=None,scope=myScope+'_conv2')
        self.conv3 = slim.convolution2d( \
            inputs=self.conv2,num_outputs=64,\
            kernel_size=[3,3],stride=[1,1],padding='VALID', \
            biases_initializer=None,scope=myScope+'_conv3')
        self.conv4 = slim.convolution2d( \
            inputs=self.conv3,num_outputs=h_size,\
            kernel_size=[7,7],stride=[1,1],padding='VALID', \
            biases_initializer=None,scope=myScope+'_conv4')
        
        self.trainLength = tf.placeholder(dtype=tf.int32)
        #We take the output from the final convolutional layer and send it to a recurrent layer.
        #The input must be reshaped into [batch x trace x units] for rnn processing, 
        #and then returned to [batch x units] when sent through the upper levles.
        self.batch_size = tf.placeholder(dtype=tf.int32,shape=[])
        self.convFlat = tf.reshape(slim.flatten(self.conv4),[self.batch_size,self.trainLength,h_size])
        self.state_in = rnn_cell.zero_state(self.batch_size, tf.float32)
        self.rnn,self.rnn_state = tf.nn.dynamic_rnn(\
                inputs=self.convFlat,cell=rnn_cell,dtype=tf.float32,initial_state=self.state_in,scope=myScope+'_rnn')
        self.rnn = tf.reshape(self.rnn,shape=[-1,h_size])
        #The output from the recurrent player is then split into separate Value and Advantage streams
        self.streamA,self.streamV = tf.split(self.rnn,2,1)
        self.AW = tf.Variable(tf.random_normal([h_size//2,a_size]))
        self.VW = tf.Variable(tf.random_normal([h_size//2,1]))
        self.Advantage = tf.matmul(self.streamA,self.AW)
        self.Value = tf.matmul(self.streamV,self.VW)
        
        self.salience = tf.gradients(self.Advantage,self.imageIn)
        #Then combine them together to get our final Q-values.
        self.Qout = self.Value + tf.subtract(self.Advantage,tf.reduce_mean(self.Advantage,axis=1,keep_dims=True))
        self.predict = tf.argmax(self.Qout,1)
        
        #Below we obtain the loss by taking the sum of squares difference between the target and prediction Q values.
        self.targetQ = tf.placeholder(shape=[None],dtype=tf.float32)
        self.actions = tf.placeholder(shape=[None],dtype=tf.int32)
        self.actions_onehot = tf.one_hot(self.actions,a_size,dtype=tf.float32)
        
        self.Q = tf.reduce_sum(tf.multiply(self.Qout, self.actions_onehot), axis=1)
        
        self.td_error = tf.square(self.targetQ - self.Q)
        
        #In order to only propogate accurate gradients through the network, we will mask the first
        #half of the losses for each trace as per Lample & Chatlot 2016
        self.maskA = tf.zeros([self.batch_size,self.trainLength//2])
        self.maskB = tf.ones([self.batch_size,self.trainLength//2])
        self.mask = tf.concat([self.maskA,self.maskB],1)
        self.mask = tf.reshape(self.mask,[-1])
        self.loss = tf.reduce_mean(self.td_error * self.mask)
        
        #Learning rate is hard-coded to 0.0001
        self.trainer = tf.train.AdamOptimizer(learning_rate=0.0001)
        self.updateModel = self.trainer.minimize(self.loss)

### Experience Replay

These classes allow us to store experies and sample then randomly to train the network.
Episode buffer stores experiences for each individal episode.
Experience buffer stores entire episodes of experience, and sample() allows us to get training batches needed from the network.

In [21]:
class experience_buffer():
    def __init__(self, trace_length = 8, buffer_size = 1000):
        self.buffer = []
        self.buffer_size = buffer_size
        self.trace_length = trace_length

        #Initialize counters and buffers for prioritixed replay
        self.episode_index = 0
        self.alpha0 = 0.5 #Start-value of alpha, the prioritized replay probability exponent. Annealing is linear to 0.
        self.alpha = self.alpha0
        self.exp_prio_tuples = []
        
    def add(self,episode):
        #Compute the sampling priority of this episode in episode replay and update the sum of priorities
        episode = np.reshape(np.array(episode),[len(episode),6])
        self.td_error = episode[:, 5]
        self.priority = np.absolute(self.td_error) + 1e-18 #proportionnal priority
        #Append episode to the priority replay tuple list
        #Every experience in the episode has a tuple of the form:
        #(episode_index, experience_index, priority, len(episode))
        for experience_index in range(self.trace_length-1, len(episode)-1):
            tup = (self.episode_index, experience_index, float(self.priority[experience_index]), len(episode))
            self.exp_prio_tuples.append(tup)
        #Need something to avoid self.exp_prio_tuples to grow infinitly
        if len(self.buffer) + 1 > self.buffer_size:
            cntr = 0
            for k in range(0, self.exp_prio_tuples[0][3]-1):
                self.exp_prio_tuples.pop(0)
                cntr += 1
            
            print('length of poped element = ' + str(len(self.buffer[0]))+ ' ,cntr = ' + str(cntr))
            self.buffer.pop(0)
            
        self.episode_index += 1
        #self.episode_index = (self.episode_index + 1) % self.buffer_size
        self.buffer.append(episode)
        
        '''
        if len(self.buffer) + 1 >= self.buffer_size:
            self.buffer[0:(1+len(self.buffer))-self.buffer_size] = []
        self.buffer.append(episode)
        '''
            
    def sample(self,batch_size):
        #Ramdomly select a number of episodes egual to batch_size 
        sampled_episodes = random.sample(self.buffer,batch_size)
        #Within the selected episodes, randomly select an experience trace of length trace_length
        sampledTraces = []
        for episode in sampled_episodes:
            point = np.random.randint(0,len(episode)+1-self.trace_length)
            sampledTraces.append(episode[point:point+self.trace_length])
        sampledTraces = np.array(sampledTraces)
        
        return np.reshape(sampledTraces,[batch_size*self.trace_length,6])
    
    def PRsample(self,batch_size):
        #alpha annealing
        self.alpha = self.alpha0 - (self.episode_index * self.alpha0/num_episodes)
        #Compute the sampling probability distribution

        
        priorities_poweralpha = np.power([tup[2] for tup in self.exp_prio_tuples],myBuffer.alpha)
        sum_priorities_poweralpha = np.sum(priorities_poweralpha)
        sampling_probabilities = np.divide(priorities_poweralpha, sum_priorities_poweralpha)
        #Sample episodes using the computed distribution
        sampled_indexes = np.random.choice(len(self.exp_prio_tuples), batch_size, p = sampling_probabilities)
        sampled_tuples = [self.exp_prio_tuples[idx] for idx in sampled_indexes]
        ep_idx = [tup[0] for tup in sampled_tuples]
        print(ep_idx)
        exp_idx = [tup[1] for tup in sampled_tuples]
        print(exp_idx)
        sampledTraces = []
        if self.episode_index <= self.buffer_size:
            idx_offset = 0
        else:
            idx_offset = self.episode_index - self.buffer_size
        print('idx_offset = ' + str(idx_offset) + ', self.episode_index = ' + str(self.episode_index) + ', len(self.buffer) = ' + str(len(self.buffer)))
        for i in range(0,batch_size):
            sampled_ep = myBuffer.buffer[ep_idx[i] - idx_offset]#does This not work?
            print('len(sampled_ep) =' + str(len(sampled_ep)))
            sampled_ep = np.reshape(np.array(sampled_ep),[len(sampled_ep),6])
            sampled_trace = sampled_ep[exp_idx[i]+1-self.trace_length:exp_idx[i]+1]
            sampledTraces.append(sampled_trace)
        sampledTraces = np.array(sampledTraces)
        print(sampledTraces.shape)
        return np.reshape(sampledTraces,[batch_size*self.trace_length,6])
    
    def save(self, path2mdl):
        #Save only last 40 experiences in buffer otherwise ridiculously large file
        np.save(path2mdl + '/experienceBuffer.npy', self.buffer[-40:])
    
    def load(self, path2mdl):
        self.buffer = list(np.load(path2mdl + '/experienceBuffer.npy'))

### Training the network

In [22]:
#Setting the training parameters
batch_size = 4 #How many experience traces to use for each training step.
trace_length = 8 #How long each experience trace will be when training
update_freq = 5 #How often to perform a training step.
y = .99 #Discount factor on the target Q-values
startE = 1 #Starting chance of random action
endE = 0.1 #Final chance of random action

prioritized_replay = True
load_model = False #Whether to load a saved model.
if load_model == True:
    last_saved_ep = 3000 #This parameter has to be updated to the last checkpoint
else:
    last_saved_ep = 0
path2mdl = "../DeepRL-Agents-Results/drqn" #The path to save our model to.
path2center = "../DeepRL-Agents-Results/Center" #The path to save the Center information to
h_size = 512 #The size of the final convolutional layer before splitting it into Advantage and Value streams.
buffer_size = 10 #Size of the episode buffer in number of episodes
max_epLength = 300 #The max allowed length of our episode.
anneling_steps = max_epLength*100 #How many steps of training to reduce startE to endE.
num_episodes = 10000 #How many episodes of game environment to train network with.
pre_train_steps = max_epLength*1 #max_epLength*100 #How many steps of random actions before training begins. need to be a multiple of max_epLength
time_per_step =  0.025 #Length of each step used in gif creation
summaryLength = 100 #Number of epidoes to periodically save for analysis
tau = 0.001 #Rate at with the target network is update in regards to the main network

In [23]:
#


#We define the cells for the primary and target q-networks

tf.reset_default_graph()

cell = tf.contrib.rnn.BasicLSTMCell(num_units=h_size,state_is_tuple=True)
cellT = tf.contrib.rnn.BasicLSTMCell(num_units=h_size,state_is_tuple=True)
mainQN = Qnetwork(h_size,cell,'main')
targetQN = Qnetwork(h_size,cellT,'target')
trainables = tf.trainable_variables()
init = tf.global_variables_initializer()
targetOps = updateTargetGraph(trainables,tau)
saver = tf.train.Saver(max_to_keep=5)

#create lists to contain total rewards and steps per episode
jList = []
rList = []


#Set the rate of random action decrease. 
e = startE
stepDrop = (startE - endE)/anneling_steps

#Make a path for our model to be saved in.
if not os.path.exists(path2mdl):
    os.makedirs(path2mdl)

with tf.Session() as sess:
    if load_model == True:
        print ('Loading Model...')
        ckpt = tf.train.get_checkpoint_state(path2mdl)
        saver.restore(sess,ckpt.model_checkpoint_path)
        #Rough (over)estimate of the total number of steps since the beginning of training
        total_steps = last_saved_ep*300/update_freq
        myBuffer = experience_buffer(trace_length, buffer_size)
        myBuffer.load(path2mdl)
    else:
        #INITIALIZE VARIABLES AND MODEL


        myBuffer = experience_buffer(trace_length,buffer_size)

        total_steps = 0
        
        sess.run(init)
        #Write the first line of the master log-file for the Control Center
        with open(path2center + '/log.csv', 'w') as myfile:
            wr = csv.writer(myfile, quoting=csv.QUOTE_ALL, lineterminator = '\n')
            wr.writerow(['Episode','Length','Reward','IMG','LOG','SAL'])   
        #Set the target network to be equal to the primary network.
        updateTarget(targetOps,sess)
    
    for i in range(last_saved_ep, num_episodes):
        #print(i)
        episodeBuffer = []
        
        #Reset environment and get first new observation
        env.new_episode()
        if use_other_buffers == True:
            st = game.get_state()
            dP = st.depth_buffer
            lP = st.labels_buffer
            sP = st.screen_buffer
            s = processBuffers(image_size, dP, lP, sP)
        else:
            sP = env.get_state().screen_buffer
            s = processImage(sP, image_size)
        d = False
        rAll = 0
        j = 0
        #Reset the recurrent layer's hidden state every episode
        state = (np.zeros([1,h_size]),np.zeros([1,h_size])) 
        #The Q-Network
        while j < max_epLength:
            
            if image_chls == 2:
                s_in = s[0:-image_size*image_size]
            else:
                s_in = s
            j+=1
            #Choose an action by greedily (with e chance of random action) from the Q-network
            if np.random.rand(1) < e or total_steps < pre_train_steps:
                #Only update the state of the RNN layer

                state1 = sess.run(mainQN.rnn_state,
                                  feed_dict={mainQN.scalarInput:[s_in/255.0],
                                             mainQN.trainLength:1, 
                                             mainQN.state_in:state,
                                             mainQN.batch_size:1})
                #Choose an action randomly
                a = np.random.randint(0,a_size)
                
            else:
                #Update the state of the RNN layer AND choose the best action
                a, state1 = sess.run([mainQN.predict,mainQN.rnn_state],
                                     feed_dict={mainQN.scalarInput:[s_in/255.0],
                                                mainQN.trainLength:1,
                                                mainQN.state_in:state,
                                                mainQN.batch_size:1})
                a = a[0]
                
            r = env.make_action(actions_list[a])
            d = env.is_episode_finished()
            if d == False:
                if use_other_buffers == True:
                    st1 = game.get_state()
                    d1P = st1.depth_buffer
                    l1P = st1.labels_buffer
                    s1P = st1.screen_buffer
                    s1 = processBuffers(image_size, d1P, l1P, s1P)
                else:
                    s1P = env.get_state().screen_buffer
                    s1 = processImage(s1P, image_size)
            else:
                break

            total_steps += 1
            
            #Compute the td error to use for prioritized replay
            if image_chls == 2:
                s1_in = s1[0:-image_size*image_size]
            else:
                s1_in = s1
            
            Q1 = sess.run(mainQN.predict,
                          feed_dict={mainQN.scalarInput:[s1_in/255.0],
                                     mainQN.trainLength:1,
                                     mainQN.state_in:state1,
                                     mainQN.batch_size:1})
                    
            Q2 = sess.run(targetQN.Qout,
                          feed_dict={targetQN.scalarInput:[s1_in/255.0],
                                     targetQN.trainLength:1,
                                     targetQN.state_in:state1,
                                     targetQN.batch_size:1})
            
            #print('Q1.shape = ' + str(Q1.shape))
            #print('Q2.shape = ' + str(Q2.shape))        
            end_multiplier = -(d - 1)
            doubleQ = Q2[0, Q1]
            #print('doubleQ.shape = ' + str(doubleQ.shape))
            targetQ = r + (y*doubleQ * end_multiplier)
            #print('targetQ.shape = ' + str(targetQ.shape))
            currentaction = np.array(a, ndmin=1)
            #print('currentaction.shape = ' + str(currentaction.shape))

            td = sess.run(mainQN.td_error,
                     feed_dict={mainQN.scalarInput:[s_in/255.0],
                                mainQN.targetQ:targetQ,
                                mainQN.actions:currentaction,
                                mainQN.trainLength:1,
                                mainQN.state_in:state,
                                mainQN.batch_size:1})
            

            
            episodeBuffer.append(np.reshape(np.array([s,a,r,s1,d,td]),[1,6]))
            
            if total_steps > pre_train_steps:
                if e > endE:
                    e -= stepDrop
                
                #Update the networks at a cetain frequency (every n experiences)
                if total_steps % (update_freq) == 0:
                    updateTarget(targetOps,sess)
                    #Reset the recurrent layer's hidden state
                    state_train = (np.zeros([batch_size,h_size]),np.zeros([batch_size,h_size])) 
                    #Get a random batch of experiences.
                    if prioritized_replay == True:
                        trainBatch = myBuffer.PRsample(batch_size)
                    else:
                        trainBatch = myBuffer.sample(batch_size)

                    train_s = list(zip(trainBatch[:, 0]))
                    train_s1 = list(zip(trainBatch[:, 3]))
                    train_s = np.vstack(train_s)
                    train_s1 = np.vstack(train_s1)

                    if image_chls == 2:
                        train_s = train_s[:,0:-image_size*image_size]
                        train_s1 = train_s1[:,0:-image_size*image_size]

                    #Below we perform the Double-DQN update to the target Q-values
                    Q1 = sess.run(mainQN.predict,
                                  feed_dict={mainQN.scalarInput:np.vstack(train_s1/255.0),
                                             mainQN.trainLength:trace_length,
                                             mainQN.state_in:state_train,
                                             mainQN.batch_size:batch_size})
                    
                    Q2 = sess.run(targetQN.Qout,
                                  feed_dict={targetQN.scalarInput:np.vstack(train_s1/255.0),
                                             targetQN.trainLength:trace_length,
                                             targetQN.state_in:state_train,
                                             targetQN.batch_size:batch_size})
                    
                    end_multiplier = -(trainBatch[:,4] - 1)
                    doubleQ = Q2[range(batch_size*trace_length),Q1]
                    targetQ = trainBatch[:,2] + (y*doubleQ * end_multiplier)
                    
                    #Update the network with our target values.
                    sess.run(mainQN.updateModel,
                             feed_dict={mainQN.scalarInput:np.vstack(train_s/255.0),
                                        mainQN.targetQ:targetQ,
                                        mainQN.actions:trainBatch[:,1],
                                        mainQN.trainLength:trace_length,
                                        mainQN.state_in:state_train,
                                        mainQN.batch_size:batch_size})
            rAll += r
            s = s1
            state = state1
            
            if use_other_buffers == True:
                lP = l1P
                dP = d1P
                sP = s1P
            else:
                sP = s1P
            

            if d == True:

                break

        #Add the episode to the experience buffer
        bufferArray = np.array(episodeBuffer)
        episodeBuffer = list(zip(bufferArray))
        myBuffer.add(episodeBuffer)
        jList.append(j)
        rList.append(rAll)

        #Periodically save the model. 
        if i % 1000 == 0 and i != last_saved_ep:
            saver.save(sess, path2mdl + '/model-'+str(i)+'.cptk', global_step = i)
            myBuffer.save(path2mdl)
            print ("Saved Model")
        if len(rList) % summaryLength == 0 and len(rList) != 0:
            #print (total_steps,np.mean(rList[-summaryLength:]), e)
            saveToCenter(i,rList,jList,
                         np.reshape(np.array(episodeBuffer),[len(episodeBuffer),6]),
                         summaryLength,
                         h_size,sess,mainQN,time_per_step,
                         image_size, image_chls, image_chls,
                         path2center)
    saver.save(sess,path2mdl + '/model-'+str(i)+'.cptk')

Target Set Success
Target Set Success
[6, 7, 0, 2]
[88, 222, 71, 251]
idx_offset = 0, self.episode_index = 10, len(self.buffer) = 10
len(sampled_ep) =270
len(sampled_ep) =299
len(sampled_ep) =268
len(sampled_ep) =294
(4, 8, 6)
Target Set Success
[9, 9, 7, 6]
[39, 31, 175, 161]
idx_offset = 0, self.episode_index = 10, len(self.buffer) = 10
len(sampled_ep) =266
len(sampled_ep) =266
len(sampled_ep) =299
len(sampled_ep) =270
(4, 8, 6)
Target Set Success
[0, 7, 9, 1]
[116, 292, 40, 240]
idx_offset = 0, self.episode_index = 10, len(self.buffer) = 10
len(sampled_ep) =268
len(sampled_ep) =299
len(sampled_ep) =266
len(sampled_ep) =299
(4, 8, 6)
Target Set Success
[1, 9, 8, 5]
[39, 20, 198, 250]
idx_offset = 0, self.episode_index = 10, len(self.buffer) = 10
len(sampled_ep) =299
len(sampled_ep) =266
len(sampled_ep) =270
len(sampled_ep) =266
(4, 8, 6)
Target Set Success
[3, 1, 0, 1]
[138, 102, 31, 11]
idx_offset = 0, self.episode_index = 10, len(self.buffer) = 10
len(sampled_ep) =270
len(sampled_e

Target Set Success
[2, 4, 3, 10]
[35, 35, 213, 234]
idx_offset = 0, self.episode_index = 11, len(self.buffer) = 11
len(sampled_ep) =294
len(sampled_ep) =270
len(sampled_ep) =270
len(sampled_ep) =282
(4, 8, 6)
Target Set Success
[6, 9, 3, 6]
[223, 60, 14, 51]
idx_offset = 0, self.episode_index = 11, len(self.buffer) = 11
len(sampled_ep) =270
len(sampled_ep) =266
len(sampled_ep) =270
len(sampled_ep) =270
(4, 8, 6)
Target Set Success
[8, 0, 10, 1]
[20, 204, 180, 67]
idx_offset = 0, self.episode_index = 11, len(self.buffer) = 11
len(sampled_ep) =270
len(sampled_ep) =268
len(sampled_ep) =282
len(sampled_ep) =299
(4, 8, 6)
Target Set Success
[2, 0, 3, 1]
[292, 170, 25, 133]
idx_offset = 0, self.episode_index = 11, len(self.buffer) = 11
len(sampled_ep) =294
len(sampled_ep) =268
len(sampled_ep) =270
len(sampled_ep) =299
(4, 8, 6)
Target Set Success
[3, 10, 3, 0]
[239, 183, 178, 40]
idx_offset = 0, self.episode_index = 11, len(self.buffer) = 11
len(sampled_ep) =270
len(sampled_ep) =282
len(samp

Target Set Success
[11, 8, 10, 6]
[222, 100, 35, 111]
idx_offset = 0, self.episode_index = 12, len(self.buffer) = 12
len(sampled_ep) =299
len(sampled_ep) =270
len(sampled_ep) =282
len(sampled_ep) =270
(4, 8, 6)
Target Set Success
[10, 7, 1, 7]
[150, 211, 215, 124]
idx_offset = 0, self.episode_index = 12, len(self.buffer) = 12
len(sampled_ep) =282
len(sampled_ep) =299
len(sampled_ep) =299
len(sampled_ep) =299
(4, 8, 6)
Target Set Success
[5, 4, 8, 10]
[139, 40, 254, 270]
idx_offset = 0, self.episode_index = 12, len(self.buffer) = 12
len(sampled_ep) =266
len(sampled_ep) =270
len(sampled_ep) =270
len(sampled_ep) =282
(4, 8, 6)
Target Set Success
[2, 5, 6, 6]
[32, 174, 25, 16]
idx_offset = 0, self.episode_index = 12, len(self.buffer) = 12
len(sampled_ep) =294
len(sampled_ep) =266
len(sampled_ep) =270
len(sampled_ep) =270
(4, 8, 6)
Target Set Success
[6, 4, 8, 0]
[38, 144, 255, 147]
idx_offset = 0, self.episode_index = 12, len(self.buffer) = 12
len(sampled_ep) =270
len(sampled_ep) =270
len(

Target Set Success
[7, 1, 10, 10]
[138, 252, 239, 196]
idx_offset = 0, self.episode_index = 12, len(self.buffer) = 12
len(sampled_ep) =299
len(sampled_ep) =299
len(sampled_ep) =282
len(sampled_ep) =282
(4, 8, 6)
Target Set Success
[5, 9, 5, 8]
[257, 166, 259, 240]
idx_offset = 0, self.episode_index = 12, len(self.buffer) = 12
len(sampled_ep) =266
len(sampled_ep) =266
len(sampled_ep) =266
len(sampled_ep) =270
(4, 8, 6)
Target Set Success
[2, 7, 3, 9]
[91, 196, 54, 51]
idx_offset = 0, self.episode_index = 12, len(self.buffer) = 12
len(sampled_ep) =294
len(sampled_ep) =299
len(sampled_ep) =270
len(sampled_ep) =266
(4, 8, 6)
Target Set Success
[2, 10, 1, 1]
[161, 241, 191, 278]
idx_offset = 0, self.episode_index = 12, len(self.buffer) = 12
len(sampled_ep) =294
len(sampled_ep) =282
len(sampled_ep) =299
len(sampled_ep) =299
(4, 8, 6)
Target Set Success
[4, 7, 7, 4]
[230, 197, 219, 80]
idx_offset = 0, self.episode_index = 12, len(self.buffer) = 12
len(sampled_ep) =270
len(sampled_ep) =299
len

Target Set Success
[8, 6, 2, 4]
[203, 102, 22, 223]
idx_offset = 0, self.episode_index = 13, len(self.buffer) = 13
len(sampled_ep) =270
len(sampled_ep) =270
len(sampled_ep) =294
len(sampled_ep) =270
(4, 8, 6)
Target Set Success
[10, 8, 2, 9]
[245, 14, 16, 222]
idx_offset = 0, self.episode_index = 13, len(self.buffer) = 13
len(sampled_ep) =282
len(sampled_ep) =270
len(sampled_ep) =294
len(sampled_ep) =266
(4, 8, 6)
Target Set Success
[8, 0, 4, 11]
[161, 197, 137, 228]
idx_offset = 0, self.episode_index = 13, len(self.buffer) = 13
len(sampled_ep) =270
len(sampled_ep) =268
len(sampled_ep) =270
len(sampled_ep) =299
(4, 8, 6)
Target Set Success
[4, 1, 1, 2]
[188, 226, 127, 200]
idx_offset = 0, self.episode_index = 13, len(self.buffer) = 13
len(sampled_ep) =270
len(sampled_ep) =299
len(sampled_ep) =299
len(sampled_ep) =294
(4, 8, 6)
Target Set Success
[3, 11, 1, 2]
[236, 30, 61, 227]
idx_offset = 0, self.episode_index = 13, len(self.buffer) = 13
len(sampled_ep) =270
len(sampled_ep) =299
len(

...

Target Set Success
[115, 127, 114, 119]
[296, 250, 95, 142]
idx_offset = 103, self.episode_index = 133, len(self.buffer) = 30
len(sampled_ep) =299
len(sampled_ep) =299
len(sampled_ep) =284
len(sampled_ep) =298
(4, 8, 6)
Target Set Success
[110, 127, 108, 117]
[253, 195, 95, 119]
idx_offset = 103, self.episode_index = 133, len(self.buffer) = 30
len(sampled_ep) =299
len(sampled_ep) =299
len(sampled_ep) =270
len(sampled_ep) =299
(4, 8, 6)
Target Set Success
[128, 119, 116, 118]
[68, 111, 110, 111]
idx_offset = 103, self.episode_index = 133, len(self.buffer) = 30
len(sampled_ep) =280
len(sampled_ep) =298
len(sampled_ep) =299
len(sampled_ep) =299
(4, 8, 6)
Target Set Success
[118, 114, 128, 114]
[104, 170, 111, 213]
idx_offset = 103, self.episode_index = 133, len(self.buffer) = 30
len(sampled_ep) =299
len(sampled_ep) =284
len(sampled_ep) =280
len(sampled_ep) =284
(4, 8, 6)
Target Set Success
[124, 115, 126, 122]
[280, 172, 66, 124]
idx_offset = 103, self.episode_index = 133, len(self.buffer

Target Set Success
[111, 127, 109, 115]
[295, 284, 235, 266]
idx_offset = 104, self.episode_index = 134, len(self.buffer) = 30
len(sampled_ep) =299
len(sampled_ep) =299
len(sampled_ep) =299
len(sampled_ep) =299
(4, 8, 6)
Target Set Success
[108, 113, 107, 115]
[41, 202, 178, 14]
idx_offset = 104, self.episode_index = 134, len(self.buffer) = 30
len(sampled_ep) =270
len(sampled_ep) =242
len(sampled_ep) =299
len(sampled_ep) =299
(4, 8, 6)
Target Set Success
[132, 112, 110, 113]
[199, 210, 276, 227]
idx_offset = 104, self.episode_index = 134, len(self.buffer) = 30
len(sampled_ep) =299
len(sampled_ep) =299
len(sampled_ep) =299
len(sampled_ep) =242
(4, 8, 6)
Target Set Success
[112, 114, 115, 117]
[188, 226, 218, 176]
idx_offset = 104, self.episode_index = 134, len(self.buffer) = 30
len(sampled_ep) =299
len(sampled_ep) =284
len(sampled_ep) =299
len(sampled_ep) =299
(4, 8, 6)
Target Set Success
[123, 131, 110, 114]
[73, 80, 249, 77]
idx_offset = 104, self.episode_index = 134, len(self.buffer)

Target Set Success
[117, 108, 107, 108]
[211, 81, 204, 228]
idx_offset = 104, self.episode_index = 134, len(self.buffer) = 30
len(sampled_ep) =299
len(sampled_ep) =270
len(sampled_ep) =299
len(sampled_ep) =270
(4, 8, 6)
Target Set Success
[132, 110, 127, 131]
[18, 267, 159, 204]
idx_offset = 104, self.episode_index = 134, len(self.buffer) = 30
len(sampled_ep) =299
len(sampled_ep) =299
len(sampled_ep) =299
len(sampled_ep) =299
(4, 8, 6)
Target Set Success
[131, 118, 115, 132]
[277, 250, 289, 197]
idx_offset = 104, self.episode_index = 134, len(self.buffer) = 30
len(sampled_ep) =299
len(sampled_ep) =299
len(sampled_ep) =299
len(sampled_ep) =299
(4, 8, 6)
Target Set Success
[108, 113, 125, 129]
[129, 31, 14, 276]
idx_offset = 104, self.episode_index = 134, len(self.buffer) = 30
len(sampled_ep) =270
len(sampled_ep) =242
len(sampled_ep) =299
len(sampled_ep) =299
(4, 8, 6)
Target Set Success
[111, 117, 114, 126]
[86, 213, 227, 293]
idx_offset = 104, self.episode_index = 134, len(self.buffer)

Target Set Success
[126, 110, 127, 127]
[131, 250, 255, 208]
idx_offset = 105, self.episode_index = 135, len(self.buffer) = 30
len(sampled_ep) =299
len(sampled_ep) =299
len(sampled_ep) =299
len(sampled_ep) =299
(4, 8, 6)
Target Set Success
[111, 128, 109, 115]
[196, 28, 36, 280]
idx_offset = 105, self.episode_index = 135, len(self.buffer) = 30
len(sampled_ep) =299
len(sampled_ep) =280
len(sampled_ep) =299
len(sampled_ep) =299
(4, 8, 6)
Target Set Success
[109, 125, 121, 119]
[115, 164, 20, 236]
idx_offset = 105, self.episode_index = 135, len(self.buffer) = 30
len(sampled_ep) =299
len(sampled_ep) =299
len(sampled_ep) =280
len(sampled_ep) =298
(4, 8, 6)
Target Set Success
[127, 117, 130, 129]
[256, 137, 267, 144]
idx_offset = 105, self.episode_index = 135, len(self.buffer) = 30
len(sampled_ep) =299
len(sampled_ep) =299
len(sampled_ep) =299
len(sampled_ep) =299
(4, 8, 6)
Target Set Success
[114, 112, 134, 131]
[93, 277, 167, 238]
idx_offset = 105, self.episode_index = 135, len(self.buffer

KeyboardInterrupt: 

In [64]:
print(type(myBuffer))
print(type(myBuffer.buffer))
print(len(myBuffer.buffer))
#myBuffer.save(path2mdl)
#myBuffer = experience_buffer()
#myBuffer.load(path2mdl)
print(type(myBuffer.buffer))
print(len(myBuffer.buffer))
print(myBuffer.buffer[98].shape)

<class '__main__.experience_buffer'>
<class 'list'>
100
<class 'list'>
100


AttributeError: 'list' object has no attribute 'shape'

In [11]:
trainBatch = myBuffer.PRsample(batch_size,trace_length, i, num_episodes)

ValueError: operands could not be broadcast together with shapes (270,1,1,6) (7056,) 

In [11]:
print(td)
episode = episodeBuffer
episode = np.reshape(np.array(episode),[len(episode),6])
print(episode.shape)
print(episode[:, 5].shape)
td_error = episode[:, 5]
priority = np.absolute(td_error) + 1e-9 #proportionnal priority
print(priority.shape)
#Append episode to the priority replay tuple list
#Every experience in the episode has a tuple of the form:
#(episode_index, experience_index, priority)
episode_index = 0

exp_prio_tuples = []
for experience_index in range(trace_length-1, len(episode[:, 5])):
    tup = (episode_index, experience_index, float(priority[experience_index]))
    exp_prio_tuples.append(tup)


print(exp_prio_tuples[2])
print(exp_prio_tuples[0][0])

priorities_poweralpha = np.power([tup[2] for tup in exp_prio_tuples],myBuffer.alpha)
print(myBuffer.alpha)
print(priorities_poweralpha.shape)
sum_priorities_poweralpha = np.sum(priorities_poweralpha)
print(sum_priorities_poweralpha)
sampling_probabilities = np.divide(priorities_poweralpha, sum_priorities_poweralpha)
print(sampling_probabilities.shape)
sampled_indexes = np.random.choice(len(exp_prio_tuples), batch_size, p = sampling_probabilities)
print(sampled_indexes)

sampled_tuples = [exp_prio_tuples[idx] for idx in sampled_indexes]
ep_idx = [tup[0] for tup in sampled_tuples]
print(ep_idx)
exp_idx = [tup[1] for tup in sampled_tuples]
print(exp_idx)
sampledTraces = []
for i in range(0,batch_size):
    sampled_ep = myBuffer.buffer[ep_idx[i]]
    sampled_ep = np.reshape(np.array(sampled_ep),[len(sampled_ep),6])
    sampled_exp = sampled_ep[exp_idx[i]-(trace_length):exp_idx[i]] 
    sampledTraces.append(sampled_exp)
sampledTraces = np.array(sampledTraces)
print(sampledTraces.shape)
result = np.reshape(sampledTraces,[batch_size*trace_length,6])

[ 0.00737692]
(21, 6)
(21,)
(21,)
(0, 9, 0.006785107310861349)
0
0.4997
(14,)
1.22897394469
(14,)
[7 2 1 8]
[0, 0, 0, 0]
[14, 9, 8, 15]
(4, 8, 6)


In [13]:
print(sampledTraces[3].shape)
print(exp_prio_tuples[0][0])
print(len(exp_prio_tuples))
exp_prio_tuples.pop(0)
print(len(exp_prio_tuples))

(8, 6)
0
14
13
